In [215]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import urllib.request
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
## ANCHORUP
log_transformer=FunctionTransformer(func=np.log,inverse_func=np.exp,validate=False,check_inverse=False)
#validate has to be false for the nans to work, as does check_inverse
#Currently not being used

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from IPython.display import Audio
def announce_when_done(target_file='https://www.myinstants.com/media/sounds/meme-de-creditos-finales.mp3'):
    '''
    place at the end of a notebook, this will play a file when done.  I have no idea how it adds to 
    notebook memory overhead.  Hopefully not much.
    '''
    return Audio(url=target_file,autoplay=True)
import missingno as msno

In [216]:
def find_overlaps(parent, child,stop_early=False,dataframe=wwdf):
    '''
    Reports if there is an element in "child" that is present in more than one "parent"
    
    "stop_early" returns at the first instance of an overlap, otherwise all overlapped-values are returned.
    '''
    check_list=dataframe.groupby(parent)[child].unique().apply(set).values.tolist()
    overlap_list=[]
    while len(check_list)>0:
        examine=check_list.pop()
        for s in examine:
            for g in check_list:
                if s in g:
                    if stop_early:
                        return f'First duplication found for {s}'
                    overlap_list.append(s)
    return set(overlap_list)
                
#way slower than it needs to be likely.
#useful for eda

In [217]:
ww=pd.read_csv(r'~\fis\tanzania_water\water_well.csv')
wt=pd.read_csv(r'~\fis\tanzania_water\water_well_target.csv')
wwdf=ww.merge(wt,on='id',validate='one_to_one') #checks to make sure we're good 

try:
    wwdf.set_index('id',inplace=True)
except:
    print('done')
    
#import both data sets, check that they work, set the indices to the id column

wwdf_raw=wwdf.copy()

In [218]:
#filling in elevation found via google maps api
try:
    wwdf['elevation']=wwdf['gps_height']
except:
    print(f'elevation setting went wrong, gps_height in columns is {"gps_height" in wwdf.columns}')

replacement_dfs=pd.read_csv('grabbed_alts_with_ids.csv',index_col=0)

wwdf.loc[replacement_dfs.index,'elevation']=replacement_dfs['elev']

#dropping the gps_height column
try:
    wwdf.drop(columns=['gps_height'],inplace=True)
except:
    print(f'something went wrong, gps_height in columns is {"gps_height" in wwdf.columns}')

In [219]:
try:
    wwdf.drop(columns='scheme_name')
except:
    print(f'elevation setting went wrong, gps_height in columns is {"scheme_name" in wwdf.columns}')
    
#Scheme name is hole-ridden and is at least partially captured by other columns, so is dropped

In [220]:
wwdf.loc[wwdf.construction_year==0,'construction_year']=np.nan
#0s are NaNs
wwdf['date_recorded']=wwdf['date_recorded'].apply(lambda d: int(d[-4:]) if type(d)==str else d)
#just extracting the year for date_recorded, conditional is present to guard against problems that emerge when re-running

In [221]:
try:
    wwdf['time_interval']=wwdf['date_recorded']-wwdf['construction_year']
    wwdf.drop(columns=['date_recorded','construction_year'],inplace=True)
except: 
    print(f'date recorded still in: {"date_recorded" in wwdf.columns}, construction year still in:  {"construction_year" in wwdf.columns}')
    
#Create time interval column -- later work may improve this

In [222]:
wwdf.public_meeting=wwdf.public_meeting.apply(lambda s: s if pd.isna(s) else (1 if s==True else 0 if s==False else s))
wwdf.permit=wwdf.permit.apply(lambda s: s if pd.isna(s) else (1 if s==True else 0 if s==False else s))
#Uglier sytnax to guard against re-running


In [223]:
for c in ['num_private','recorded_by','wpt_name']:
    try:     
        wwdf.drop(columns=[c],inplace=True)
    except:
        print(f'recorded_by still in: {"recorded_by" in wwdf.columns}, wpt_name still in:  {"wpt_name" in wwdf.columns}, num_private still in: {"num_private" in wwdf.columns}')

      #not useful columns

In [224]:
wwdf=wwdf.applymap(lambda s: s.upper() if type(s)==str else s)
#Crude protection by way of 

In [229]:
try:
    amount_tsh_safeguard+=1
    print('did not run, protected')
except:
    amount_tsh_safeguard=0
if amount_tsh_safeguard==0:
    wwdf['amount_tsh']=wwdf['amount_tsh'].apply(lambda n: np.log(n+1))

#mapping amount tsh to log(n+1) (+1 to guard against the 0s), which is inside a conditional meant to protect it from being re-ran mindlessly

did not run, protected


In [230]:
wwdf['population']=wwdf['population'].apply(lambda x: np.nan if (x==0 or x==1) else x)
#Mapping nan values for population

In [231]:
wwdf['water_quality_character']=wwdf.water_quality.apply(lambda s: s.replace(' ABANDONED',''))
wwdf['water_quality_abandoned']=wwdf.water_quality.apply(lambda s: 1 if 'ABANDONED' in s else 0)

#Another way to look at water_quality

In [232]:
# Mostly ordered in terms of apparent quality, except for location
# columns that seem to be about roughly the same thing
# treating long&lat&elevation separate from other location stuff for syntactical reasons
# similarly, water_quality_character&water_quality_abandoned are not in here as they should be used together
exclusives=[['basin','subvillage','region','region_code','district_code','lga','ward'],
 ['scheme_management','scheme_name'],
 ['extraction_type_group','extraction_type_class','extraction_type'],
 ['management','management_group'],
 ['payment_type','payment'],
 ['water_quality','quality_group'],
 ['quantity','quantity_group'],
 ['source_class','source','source_type'],
 ['waterpoint_type','waterpoint_type_group']
]

be_wary=['population','installer','subvillage','lga']

notes='subvillage is large, lga is large'

### This is-- in principle-- a leaky method that's being used right now.  Should go back and try to make a non-leaky filling with KNN for lat&long,
### notice this will require going back and changing how elevation is handled
### It shouldn't *really* be leaky though, since we're not assuming some distribution *really*, each 
### location does have a specific area and that's what we're mapping from.  We could be **more** accurate with more data, but I don't think overfitting should really be possible like this. 

In [233]:

subvillage_dict=wwdf.loc[(wwdf.latitude!=-2e-08) & (wwdf.longitude!=0)].groupby('subvillage')[['longitude','latitude']].mean().to_dict(orient='index')
ward_dict=wwdf.loc[(wwdf.latitude!=-2e-08) & (wwdf.longitude!=0)].groupby('ward')[['longitude','latitude']].mean().to_dict(orient='index')
lga_dict=wwdf.loc[(wwdf.latitude!=-2e-08) & (wwdf.longitude!=0)].groupby('lga')[['longitude','latitude']].mean().to_dict(orient='index')
region_code_dict=wwdf.loc[(wwdf.latitude!=-2e-08) & (wwdf.longitude!=0)].groupby('region_code')[['longitude','latitude']].mean().to_dict(orient='index')
def guess_latlong(row):
    try:
        lat=subvillage_dict[row['subvillage']]['latitude']
        long=subvillage_dict[row['subvillage']]['longitude']
        return(lat,long)
    except:
        
#         print('subvillage failed')
        pass
    try:
        lat=ward_dict[row['subvillage']]['latitude']
        long=ward_dict[row['ward']]['longitude']
        return(lat,long)
    except:
#         print('ward failed')
        pass
    try:
        lat=lga_dict[row['lga']]['latitude']
        long=lga_dict[row['lga']]['longitude']
        return(lat,long)
    except:
#         print('lgafailed')
        pass
    try:
        lat=region_code_dict[row['region_code']]['latitude']
        long=region_code_dict[row['region_code']]['longitude']
        return(lat,long)

    except:
        print(f'error for {row.name}')
        return(row.latitude,row.longitude)
print(guess_latlong(wwdf.loc[6091]))

(-2.962380523139534, 34.179475708779066)


In [234]:
bad_vals=wwdf.loc[(wwdf['longitude']==0) | ((wwdf['latitude']==-2e-08)),['latitude','longitude']]
display(bad_vals.head())
new_val_series=wwdf.loc[(wwdf['longitude']==0) | ((wwdf['latitude']==-2e-08))].apply(lambda r: guess_latlong(r),axis=1)
try:
    new_vals=pd.DataFrame([[LAT,LONG] for LAT,LONG in new_val_series],columns=['latitude','longitude'],index=bad_vals.index)
    wwdf.loc[(wwdf['longitude']==0) | ((wwdf['latitude']==-2e-08)),['latitude','longitude']]=new_vals
except: 
    print('might be done already')
display(new_vals.head(3))

print('checking latitude value counts now:')
print((wwdf.latitude.value_counts()).head(3))

,latitude,longitude
id,,
6091,-2.000000e-08,0.0
32376,-2.000000e-08,0.0
72678,-2.000000e-08,0.0
56725,-2.000000e-08,0.0
13042,-2.000000e-08,0.0


,latitude,longitude
id,,
6091,-2.962381,34.179476
32376,-2.359152,34.535098
72678,-2.962381,34.179476


checking latitude value counts now:
-2.962381    696
-2.620502    278
-2.568506    164
Name: latitude, dtype: int64


In [235]:
wwdf['extraction_type']=wwdf['extraction_type'].apply(lambda s: s.replace('OTHER - ',''))
#cleaning up the formatting

# Some time-intervals  are wrong, and are probably??? a result of an incorrect date_recorded, likely 2004 should be 2014?  Waiting for feedback from Alex though, for now just dropping them:

In [243]:
wwdf.loc[wwdf.time_interval<0,'time_interval']=np.nan

# One final look-over before exporting the data:

In [246]:
for c in wwdf:
    print(f'{c}: missing values: {wwdf[c].isna().sum()}\nunique values:{wwdf[c].nunique()}')
    display(wwdf[c].value_counts(normalize=True,dropna=False))
    print('---'*10)

amount_tsh: missing values: 0
unique values:98


0.0         0.700993
500.0       0.052222
50.0        0.041616
1000.0      0.025051
20.0        0.024630
              ...   
8500.0      0.000017
6300.0      0.000017
220.0       0.000017
138000.0    0.000017
12.0        0.000017
Name: amount_tsh, Length: 98, dtype: float64

------------------------------
funder: missing values: 3635
unique values:1897


GOVERNMENT OF TANZANIA    0.152929
NaN                       0.061195
DANIDA                    0.052424
HESAWA                    0.037071
RWSSP                     0.023131
                            ...   
KAHEMA                    0.000017
MISRI GOVERNMENT          0.000017
MSIKITINI                 0.000017
MAKAPUCHINI               0.000017
MATATA SELEMANI           0.000017
Name: funder, Length: 1898, dtype: float64

------------------------------
installer: missing values: 3655
unique values:1935


...

------------------------------
water_quality_character: missing values: 0
unique values:6


SOFT        0.855522
SALTY       0.087458
UNKNOWN     0.031582
MILKY       0.013535
COLOURED    0.008249
FLUORIDE    0.003653
Name: water_quality_character, dtype: float64

------------------------------
water_quality_abandoned: missing values: 0
unique values:2


0    0.994007
1    0.005993
Name: water_quality_abandoned, dtype: float64

------------------------------


In [249]:
wwdf.to_csv('cleaned_data_072920_shaw.csv')

In [250]:
pd.read_csv('cleaned_data_072920_shaw.csv')

,id,amount_tsh,funder,installer,longitude,latitude,basin,subvillage,region,region_code,...,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,elevation,time_interval,water_quality_character,water_quality_abandoned
0,69572,6000.0,ROMAN,ROMAN,34.938093,-9.856322,LAKE NYASA,MNYUSI B,IRINGA,11,...,SPRING,SPRING,GROUNDWATER,COMMUNAL STANDPIPE,COMMUNAL STANDPIPE,FUNCTIONAL,1390.000000,12.0,SOFT,0
1,8776,0.0,GRUMETI,GRUMETI,34.698766,-2.147466,LAKE VICTORIA,NYAMARA,MARA,20,...,RAINWATER HARVESTING,RAINWATER HARVESTING,SURFACE,COMMUNAL STANDPIPE,COMMUNAL STANDPIPE,FUNCTIONAL,1399.000000,3.0,SOFT,0
2,34310,25.0,LOTTERY CLUB,WORLD VISION,37.460664,-3.821329,PANGANI,MAJENGO,MANYARA,21,...,DAM,DAM,SURFACE,COMMUNAL STANDPIPE MULTIPLE,COMMUNAL STANDPIPE,FUNCTIONAL,686.000000,4.0,SOFT,0
3,67743,0.0,UNICEF,UNICEF,38.486161,-11.155298,RUVUMA / SOUTHERN COAST,MAHAKAMANI,MTWARA,90,...,MACHINE DBH,BOREHOLE,GROUNDWATER,COMMUNAL STANDPIPE MULTIPLE,COMMUNAL STANDPIPE,NON FUNCTIONAL,263.000000,27.0,SOFT,0
4,19728,0.0,ACTION IN A,ARTISAN,31.130847,-1.825359,LAKE VICTORIA,KYANYAMISA,KAGERA,18,...,RAINWATER HARVESTING,RAINWATER HARVESTING,SURFACE,COMMUNAL STANDPIPE,COMMUNAL STANDPIPE,FUNCTIONAL,1671.699341,NaN,SOFT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,GERMANY REPUBLI,CES,37.169807,-3.253847,PANGANI,KIDURUNI,KILIMANJARO,3,...,SPRING,SPRING,GROUNDWATER,COMMUNAL STANDPIPE,COMMUNAL STANDPIPE,FUNCTIONAL,1210.000000,14.0,SOFT,0
59396,27263,4700.0,CEFA-NJOMBE,CEFA,35.249991,-9.070629,RUFIJI,IGUMBILO,IRINGA,11,...,RIVER,RIVER/LAKE,SURFACE,COMMUNAL STANDPIPE,COMMUNAL STANDPIPE,FUNCTIONAL,1212.000000,15.0,SOFT,0
59397,37057,0.0,NaN,NaN,34.017087,-8.750434,RUFIJI,MADUNGULU,MBEYA,12,...,MACHINE DBH,BOREHOLE,GROUNDWATER,HAND PUMP,HAND PUMP,FUNCTIONAL,1069.052612,NaN,FLUORIDE,0
59398,31282,0.0,MALEC,MUSA,35.861315,-6.378573,RUFIJI,MWINYI,DODOMA,1,...,SHALLOW WELL,SHALLOW WELL,GROUNDWATER,HAND PUMP,HAND PUMP,FUNCTIONAL,1053.703613,NaN,SOFT,0
